In [1]:
from unidec_modules.ChromEng2 import *
import plate_map2 as pm2
from dataclasses import dataclass
import pandas as pd

In [2]:
spectra_path = "C:\\Users\\cm19ljc\\Documents\\GitHub\\UniDec\\mzML\\mzML.mzML"
map_path = "unichrom plate map example.xlsx"

In [3]:
eng = ChromEngine()
eng.load_mzml(spectra_path)
eng.config.chrom_peak_width = 0.05
eng.get_chrom_peaks() # INSERT PLOT HERE


UniDec Engine v.4.4.0

UniDec Path: C:\Users\cm19ljc\Documents\GitHub\UniDec\unidec_bin\UniDec.exe

UniDec Engine v.4.4.0

UniDec Path: C:\Users\cm19ljc\Documents\GitHub\UniDec\unidec_bin\UniDec.exe
Opening HDF5 File: C:\Users\cm19ljc\Documents\GitHub\UniDec\mzML\mzML.hdf5
Reading mzML: C:\Users\cm19ljc\Documents\GitHub\UniDec\mzML\mzML.mzML
[Warning] Not index found and build_index_from_scratch is False
Converted to gzip file to improve speed: C:\Users\cm19ljc\Documents\GitHub\UniDec\mzML\mzML.mzML.gz
Error getting TIC in mzML; trying to make it...
Imported Data. Constructing TIC
Done
1070 2
Noise Level: 0.03357343719702872 Window: 0.05
Bad Peak [0.419567   0.30422294] 0.23768299999999998 [0.419567, 0.65725]
Bad Peak [0.602767   0.24142904] 0.24183299999999996 [0.419567, 0.6614]
0.702967 0.14128400000000008
0.790233 0.16205000000000003
1.0312 0.13711000000000007
1.95747 0.1411800000000003
2.88835 0.13712999999999997
Bad Peak [3.76958    0.03455761] 0.8769500000000003 [3.7197, 4.59665

([array([0.702967  , 0.13838679]),
  array([0.790233  , 0.09750636]),
  array([1.0312    , 0.74873995]),
  array([1.95747   , 0.88543616]),
  array([2.88835, 1.     ])],
 [[0.686333, 0.827617],
  [0.682183, 0.844233],
  [1.00212, 1.13923],
  [1.9284, 2.06958],
  [2.85927, 2.9964]])

In [32]:
TIC_min = 1 # ADD TO PACKAGE
tranges = np.array(eng.chrompeaks_tranges)
eng.chrompeaks_tranges = tranges[np.all(tranges > 1, axis = 1)] # INSERT PLOT HERE
eng.chrompeaks_tranges

array([[1.00212, 1.13923],
       [1.9284 , 2.06958],
       [2.85927, 2.9964 ]])

list

In [30]:
tranges = np.array(tranges)
tranges[np.all(tranges > 1, axis = 1)]

array([[1.00212, 1.13923],
       [1.9284 , 2.06958],
       [2.85927, 2.9964 ]])

In [5]:
# eng.config.sw_scan_offset = 0 # ??? defaults to 10

eng.add_chrom_peaks2() # stores data from selected peaks ready for metaunidec processing

Getting scans: [98, 131]
Length merge axis: 28915
Getting scans: [321, 355]
Length merge axis: 30293
Getting scans: [545, 578]
Length merge axis: 31336


In [6]:
eng.config.subtype = 2 # background subtraction - subtract curved
eng.config.subbuff = 100 # background subtraction amount(subtract curved) 0 = 0ff, 100 = good amount when on
eng.config.datanorm = 0 # turn off data normalisation

# -- Deconvolution
eng.config.numit = 100 # number of iterations

# mass range (default = 5000.0 to 500000.0 Da)
eng.config.massub = 15000 # upper 
eng.config.masslb = 11000 # lower

eng.config.massbins = 0.1 # sample mass every 0.1 Da

# FWHM 
# eng.get_auto_peak_width()
eng.config.mzsig = 0 

# charge range
eng.config.startz = 1
eng.config.endz = 30

# smoothing 
eng.config.zzsig = 1 # charge smooth width (smooth charge state distributions)
eng.config.psig = 1 # smooth nearby points (point smooth width, some = 1)
eng.config.beta = 0 # suppress artifacts (beta, some = 50)

eng.config.psfun = 0 # Peak shape function (gaussian, lorentzian, split G/L)

# -- Peak Selection and plotting
eng.config.peaknorm = 0 # Normalise peaks (0 = off)
eng.config.datanorm = 0
eng.config.peakwindow = 10 # peak window / Da
eng.config.exnorm = 0 # extract normalisation
eng.config.peakthresh = 0.05
# eng.config.nativeub = 10
# eng.config.nativelb = -10
eng.data.export_hdf5()

In [7]:
eng.process_data()
eng.run_unidec()
eng.pick_peaks() 

Execution Time: 0.04523450000000029
Execution Time: 1.4934326000000002
Execution Time: 0.0678011999999999
Peak info saved to: C:\Users\cm19ljc\Documents\GitHub\UniDec\mzML\UniDec_Figures_and_Files\mzML_extracts.txt


In [8]:
# PLATE MAP PROCESSING STARTS HERE
rmap, pmap = pm2.read_in_long(map_path)
eng = pm2.update_vars(eng, pmap, skip_empty = True)
print(rmap)
print(pmap)

        Reaction  Species  Concentration Units     Mass Reagent_Type Sequence
0  17-05-21-CTB1      CTB           10.0    uM  13321.0           SM      NaN
1  17-05-21-CTB1  GVYQSRL           10.0    uM  13184.0           SM  GVYQSRL
2  17-05-21-CTB1     HC7D           10.0    uM  13595.0            C      NaN
3  17-05-21-CTB1   CTB-GV            NaN    uM  13627.0            P      NaN
     Type       Reaction Time  Valid
A1   Test  17-05-21-CTB1    0   True
A2   Test  17-05-21-CTB1   30   True
A3   Test  17-05-21-CTB1   60   True
B1  empty            NaN  NaN   True
B2  empty            NaN  NaN   True
B3  empty            NaN  NaN   True


In [9]:
class Species:
    def __init__(self, dictionary, name, peak = None):
        self.__dict__.update(dictionary)
        self.__name__ = name
        self.peak = peak
        
    
    def __repr__(self):
        keys = [key for key, val in self.__dict__.items()]
        vals = [val for key, val in self.__dict__.items()]
        return self.__name__+ "("+", ".join("{} = {}".format(*t) for t in zip(keys, vals))+")"        

In [10]:
class Time:
    def __init__(self, time, species, coord = None, name = 'Time'):
        self.species = species # list of species in each time point
        self.time = time
        self.coord = coord
        self.__name__= name
        self.spectra = None
        self.thresh = 1
    
    def __repr__(self):
        species_str = "species = (" + ", ".join(s.__name__ for s in self.species)+")"
        if self.spectra == None:
            return self.__name__+ "(time = {}, coord = {}, ".format(self.time, self.coord) + species_str + ")" 
        else:
            return self.__name__+ "(time = {}, coord = {}, spectra = {}, ".format(self.time, self.coord, self.spectra)+ species_str + ")" 
        
    def extract_masses(self):
        self.theory_masses = np.array([sp.Mass for sp in self.species])
        self.species_name = np.array([sp.__name__ for sp in self.species], dtype = str)
        self.data_masses= np.array([p.mass for p in self.spectra.pks.peaks])
        self.pks = np.array([p for p in self.spectra.pks.peaks])
    def match_masses(self, thresh = None):
        pass
#         if thresh != None:
#             self.thresh = thresh
#         x, y = np.meshgrid(self.data_masses, self.theory_masses)
#         diff = abs(x - y)
#         diff[diff>self.thresh] = np.nan
#         try:
#             mins = np.nanargmin(diff, axis = 0)
#             self.peaks = self.theory_masses[mins]
#         except:
#             mins = [np.nanargmin(i) for i in diff if np.isnan(i).all() == False]
#             self.peaks = self.theory_masses[mins]

In [11]:
reactions_species = {}
groupbyparam = 'Time'
reactions_groupby = {}
for rkey, rval in rmap.groupby(['Reaction']):
    reactions_species[rkey] = [Species(spval.to_dict('records')[0], 
                                name = spkey) for spkey, spval in rval.groupby('Species')]
        
for rkey, rval in pmap.groupby('Reaction'):
    # update each species with time points and coordinates
    for species in reactions_species[rkey]:
        setattr(species, groupbyparam, rval[groupbyparam].values)
        setattr(species, "wells", np.array(rval.index))
        
    time = rval[groupbyparam].values
    coords = np.array(rval[groupbyparam].index)
    reactions_groupby[rkey] = [Time(species = reactions_species[rkey], 
                                  time = t, coord = c) for t, c in zip(time, coords)]


In [12]:
spectra_dict = {s.var1:s for s in eng.data.spectra}
spectra_dict

{'A1': <metaunidec.mudstruct.Spectrum at 0x1b464c7de20>,
 'A2': <metaunidec.mudstruct.Spectrum at 0x1b464c7dfa0>,
 'A3': <metaunidec.mudstruct.Spectrum at 0x1b464c7d8e0>}

In [13]:
times = []
for k, rval in reactions_groupby.items():
    for t in rval:
        t.spectra = spectra_dict[t.coord]
#         print(t)
        t.extract_masses()
        print(t.species_name)
        print(t.theory_masses)
        print(t.data_masses)
        print("--------------------------------")
        times.append(t)

['CTB' 'CTB-GV' 'GVYQSRL' 'HC7D']
[13321. 13627. 13184. 13595.]
[13047.7 13184.7 13321.7 13595.4 13627.  13659.9]
--------------------------------
['CTB' 'CTB-GV' 'GVYQSRL' 'HC7D']
[13321. 13627. 13184. 13595.]
[13047.101 13184.301 13321.5   13595.601 13627.801]
--------------------------------
['CTB' 'CTB-GV' 'GVYQSRL' 'HC7D']
[13321. 13627. 13184. 13595.]
[13046.7 13184.7 13321.6 13595.6 13627.3]
--------------------------------


In [14]:
t = times[0]

t.species_name

array(['CTB', 'CTB-GV', 'GVYQSRL', 'HC7D'], dtype='<U7')

In [15]:
t.theory_masses

array([13321., 13627., 13184., 13595.])

In [16]:
t.data_masses

array([13047.7, 13184.7, 13321.7, 13595.4, 13627. , 13659.9],
      dtype=float32)

In [17]:
x, y = np.meshgrid(t.theory_masses, t.data_masses)
x

array([[13321., 13627., 13184., 13595.],
       [13321., 13627., 13184., 13595.],
       [13321., 13627., 13184., 13595.],
       [13321., 13627., 13184., 13595.],
       [13321., 13627., 13184., 13595.],
       [13321., 13627., 13184., 13595.]])

In [18]:
y

array([[13047.7, 13047.7, 13047.7, 13047.7],
       [13184.7, 13184.7, 13184.7, 13184.7],
       [13321.7, 13321.7, 13321.7, 13321.7],
       [13595.4, 13595.4, 13595.4, 13595.4],
       [13627. , 13627. , 13627. , 13627. ],
       [13659.9, 13659.9, 13659.9, 13659.9]], dtype=float32)

In [19]:
diff = abs(x-y)
diff

array([[2.73299805e+02, 5.79299805e+02, 1.36299805e+02, 5.47299805e+02],
       [1.36299805e+02, 4.42299805e+02, 7.00195312e-01, 4.10299805e+02],
       [7.00195312e-01, 3.05299805e+02, 1.37700195e+02, 2.73299805e+02],
       [2.74400391e+02, 3.15996094e+01, 4.11400391e+02, 4.00390625e-01],
       [3.06000000e+02, 0.00000000e+00, 4.43000000e+02, 3.20000000e+01],
       [3.38900391e+02, 3.29003906e+01, 4.75900391e+02, 6.49003906e+01]])

In [20]:
diff[diff>10]=np.nan
diff

array([[       nan,        nan,        nan,        nan],
       [       nan,        nan, 0.70019531,        nan],
       [0.70019531,        nan,        nan,        nan],
       [       nan,        nan,        nan, 0.40039062],
       [       nan, 0.        ,        nan,        nan],
       [       nan,        nan,        nan,        nan]])

In [21]:
# check this works with more species than data peaks 
try:
    mins = np.nanargmin(diff, axis = 0)
    peaks = t.species_name[mins]
except:
    peak_dict = {}
    for i, d in enumerate(diff):
        if np.isnan(d).all()==False:
            
            mini = np.nanargmin(d)
            peak_species = t.species_name[mini]
            
            data_peak = t.data_masses[i]
            t.species[mini].peak = t.pks[i]
            peak_dict[peak_species] = data_peak
            
#     mins = [np.nanargmin(i) for i in diff if np.isnan(i).all() == False]
#     peaks = t.species_name[mins]
            
#     print(species)
# print(peaks)
# print(t.data_masses)
# print("----------------------------")
# print(t.species_name)
# print(t.theory_masses)
peak_dict

{'GVYQSRL': 13184.7, 'CTB': 13321.7, 'HC7D': 13595.4, 'CTB-GV': 13627.0}

In [22]:
t.pks

array([<unidec_modules.peakstructure.Peak object at 0x000001B411BB6D00>,
      dtype=object)

In [23]:
for s in t.species:
    print(s.__name__, s.Mass, s.peak.mass)

CTB 13321.0 13321.7
CTB-GV 13627.0 13627.0
GVYQSRL 13184.0 13184.7
HC7D 13595.0 13595.4
